In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('label_ner.csv')

In [3]:
data

,Sentence #,Word,POS,Tag
0,0,hàng_chính_hãng,N,B-OBJ
1,0,flagship,N,B-ORG
2,0,store,N,O
3,0,hồng_siêu_iu,V,O
4,0,gói_hàng,N,B-OBJ
...,...,...,...,...
134194,23873,bắt,V,O
134195,23874,hàng,N,B-OBJ
134196,23874,check,V,O
134197,23874,hàng,N,B-OBJ


In [7]:
grouped = data.groupby('Sentence #')
# Tạo danh sách các câu đã được kết hợp từ và tag
sentences = []
tags = []
for name, group in grouped:
    sentence = ' '.join(group['Word'])
    tag_sequence = ' '.join(group['Tag'])
    sentences.append(sentence)
    tags.append(tag_sequence)

# Tạo DataFrame mới cho kết quả
result = pd.DataFrame({
    'text': sentences,
    'labels': tags
})


In [8]:
result

,text,labels
0,hàng_chính_hãng flagship store hồng_siêu_iu gó...,B-OBJ B-ORG O O B-OBJ O B-OBJ B-ADJ B-ADJ
1,chất_lượng giao_hàng nhanh sản_phẩm đúng chính...,B-ADJ B-OBJ B-ADJ B-OBJ O B-OBJ O O O B-OBJ B-OBJ
2,chuẩn giao_hàng nhanh hình_ảnh tính_chất,O B-OBJ B-ADJ B-OBJ B-OBJ
3,giao_hàng đúng hẹn chất_lượng máy tốt,B-OBJ O O B-ADJ O O
4,hồng cưng xiu,B-OBJ O O
...,...,...
23823,qa tốt,O O
23824,cay cú,O O
23825,iphone khuyên iphone,B-OBJ O B-OBJ
23826,bắt,O


In [10]:
result.to_csv('ner.csv', index=False, encoding='utf-8-sig')


In [9]:
import sys
import warnings

import pandas
import torch
from numpy import VisibleDeprecationWarning
from sentence_transformers import SentenceTransformer
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import classification_report
from torch import nn
from torch import from_numpy
from torch.nn.functional import softmax
from transformers import XLNetTokenizer, T5Tokenizer, GPT2Tokenizer

In [22]:
import sys
import warnings
from transformers import AutoModel, AutoTokenizer

import numpy
import pandas
import torch
from numpy import VisibleDeprecationWarning
from sentence_transformers import SentenceTransformer
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import classification_report
from torch import nn
from torch import from_numpy
from torch.nn.functional import softmax
from transformers import XLNetTokenizer, T5Tokenizer, GPT2Tokenizer

# Add these to silence the warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=VisibleDeprecationWarning)


def get_label2id_vocab(data):
    label2idx = {'PAD': 0, 'UNK': 1}
    idx = len(label2idx)
    for line in data:
        for l in line:
            if l not in label2idx:
                label2idx[l] = idx
                idx += 1
    return label2idx


def get_label_ids(data, labels2ids):
    labels_ids = []
    for line in data:
        label_ids = []
        for l in line:
            label_ids.append(labels2ids.get(l, labels2ids.get('UNK')))
        labels_ids.append(label_ids)
    return labels_ids


class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_labels, dropout):
        super(Classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_labels = num_labels

        self.dropout = nn.Dropout(dropout)
        self.ff = nn.Linear(self.embedding_dim, num_labels)

    def forward(self, inputs):
        tensor = self.dropout(inputs)
        tensor = self.ff(tensor)
        tensor = tensor.view(-1, self.num_labels)
        probs = softmax(tensor, dim=-1)
        return tensor, probs


class Batcher(object):
    def __init__(self, data_x, data_y, batch_size,
                 emb_dim, pad_id, pad_id_labels, seed=0):
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
        self.num_samples = data_x.shape[0]
        self.emb_dim = emb_dim
        self.indices = numpy.arange(self.num_samples)
        self.ptr = 0
        self.rnd = numpy.random.RandomState(seed)
        self.pad_id = pad_id
        self.pad_id_labels = pad_id_labels

    def __iter__(self):
        return self

    def __next__(self):
        if self.ptr >= self.num_samples:
            self.ptr = 0
            self.rnd.shuffle(self.indices)
            raise StopIteration
        else:
            batch_x = self.data_x[self.ptr: self.ptr + self.batch_size]
            batch_y = self.data_y[self.ptr: self.ptr + self.batch_size]
            self.ptr += self.batch_size

            lengths = [len(x) for x in batch_x]
            max_length_batch = max(lengths)

            paddings = torch.full((max_length_batch * self.emb_dim,),
                                  fill_value=self.pad_id,
                                  dtype=torch.float32,
                                  )
            padded_labels = self.pad_id_labels * numpy.ones((len(batch_y), max_length_batch))
            all_emb = list()
            for idx, embeddings in enumerate(batch_x):
                curr_len = embeddings.shape[0]
                padded_labels[idx][:curr_len] = batch_y[idx]

                all_emb += [emb for emb in embeddings]
                num_pads = max_length_batch - curr_len

                if num_pads > 0:
                    t = paddings[
                        :num_pads * self.emb_dim
                    ]
                    all_emb.append(t)
            batch_x_final = torch.cat(all_emb).view(
                [len(batch_x), max_length_batch, self.emb_dim]
            )

            batch_y_final = torch.LongTensor(padded_labels)

            return batch_x_final, batch_y_final


data_df = pandas.read_csv('ner.csv')
print(data_df.head())
sentences = data_df.text.tolist()
labels = data_df.labels.tolist()
sentences_filtered = []
labels_filtered = []
for text, label in zip(sentences, labels):
    if len(text.split()) == len(label.split()):
        sentences_filtered.append(text)
        labels_filtered.append(label)
    else:
        sys.stderr.write(f'Ignored example: {text} {label}\n')


labels_filtered = [l.split() for l in labels_filtered]
train_sentences = sentences_filtered[0:19062]
train_labels = labels_filtered[0:19062]

test_sentences = sentences_filtered[19062:23828]
test_labels = labels_filtered[4500:23828]

labels2idx = get_label2id_vocab(train_labels)
idx2labels = {labels2idx[key]: key for key in labels2idx}

train_labels = get_label_ids(train_labels, labels2idx)
test_labels = get_label_ids(test_labels, labels2idx)

class PhoBERTWrapper:
    def __init__(self, model_name='vinai/phobert-base', device='cpu'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = device
        self.model.to(self.device)
    
    def encode(self, sentences, output_value='token_embeddings', show_progress_bar=True, convert_to_tensor=False, device='cpu'):
        self.model.to(device)
        if show_progress_bar:
            from tqdm import tqdm
            sentences = tqdm(sentences)
        
        encoded = []
        for sentence in sentences:
            inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True).to(device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            if output_value == 'token_embeddings':
                # Get the hidden states from the last layer
                last_hidden_states = outputs.last_hidden_state
                if convert_to_tensor:
                    last_hidden_states = last_hidden_states.detach()
                encoded.append(last_hidden_states.cpu().numpy() if not convert_to_tensor else last_hidden_states)
        return encoded
    
encoder = PhoBERTWrapper()

train_embeddings = encoder.encode(train_sentences,
                                  output_value='token_embeddings',
                                  show_progress_bar=True,
                                  convert_to_tensor=False,
                                  device='cpu')

test_embeddings = encoder.encode(test_sentences,
                                 output_value='token_embeddings',
                                 show_progress_bar=True,
                                 convert_to_tensor=False,
                                 device='cpu')

tokenizer = encoder.tokenizer

# Giả sử tất cả các tokenizer khác đều bắt đầu bằng một token đặc biệt
begin_offset = 1 if 'cls_token_id' in tokenizer.special_tokens_map else 0

# # Most models have an initial BOS/CLS token, except for XLNet, T5 and GPT2
# begin_offset = 1
# if type(tokenizer) == XLNetTokenizer:
#     begin_offset = 0
# if type(tokenizer) == T5Tokenizer:
#     begin_offset = 0
# if type(tokenizer) == GPT2Tokenizer:
#     begin_offset = 0


def subword2word_embeddings(subword_embeddings, text):
    sentence_embeddings = []
    pooling = 'mean'
    for embedding, sentence in zip(subword_embeddings, text):
        start_sub_token = begin_offset
        token_embeddings = []
        for word in sentence.split():
            sub_tokens = tokenizer.tokenize(word)
            num_sub_tokens = len(sub_tokens)
            all_embeddings = embedding[start_sub_token:start_sub_token + num_sub_tokens]

            if not isinstance(all_embeddings, torch.Tensor):
                # Convert numpy embeddings to tensor
                all_embeddings = from_numpy(all_embeddings)

            if pooling == 'mean':
                final_embeddings = torch.mean(all_embeddings, dim=0)
            elif pooling == 'last':
                final_embeddings = all_embeddings[-1]
            else:
                final_embeddings = all_embeddings[0]

            token_embeddings.append(final_embeddings)
            start_sub_token += num_sub_tokens

        sentence_embeddings.append(torch.stack(token_embeddings))

    return sentence_embeddings


                                                text  \
0  hàng_chính_hãng flagship store hồng_siêu_iu gó...   
1  chất_lượng giao_hàng nhanh sản_phẩm đúng chính...   
2           chuẩn giao_hàng nhanh hình_ảnh tính_chất   
3              giao_hàng đúng hẹn chất_lượng máy tốt   
4                                      hồng cưng xiu   

                                              labels  
0          B-OBJ B-ORG O O B-OBJ O B-OBJ B-ADJ B-ADJ  
1  B-ADJ B-OBJ B-ADJ B-OBJ O B-OBJ O O O B-OBJ B-OBJ  
2                          O B-OBJ B-ADJ B-OBJ B-OBJ  
3                                B-OBJ O O B-ADJ O O  
4                                          B-OBJ O O  


Ignored example: điện_thoại tiên đắt_điện_tử camera đập hộp vui vui B-OBJ O B-OBJ O O O O
Ignored example: giàu vitamin khoáng chất tốt_hệ miễn_dịch đóng_gói_tiện lợi O O O O O O B-OBJ
Ignored example: perfect chức_năng multifunction bé hồng xinh_hộp mát_lòng mát_lazada giao_hàng nhanh_đóng_gói kĩ_shipper thân_thiện hài_lòng máy_toanh nguyên_tem nhãn O O O O O O O B-OBJ O O O O O O B-OBJ
Ignored example: hàng tiên quà tiên_điện_thoại lazada lắng yên_lớp niêm_phong_giá tốt mê B-OBJ O O O B-ORG O O O O
Ignored example: chất_lượng sản_phẩm tuyệt_vời đóng_gói cẩn_thận vận hàng nhanh ủng_hộ B-ADJ B-OBJ O B-OBJ O B-OBJ B-ADJ O
Ignored example: sản_phẩm đúng mô_tả giao_hàng nhanh_mã nguyên đai_nguyện kiện B-OBJ O O B-OBJ O O O
Ignored example: tính_năng tốt tính_năng công_dụng chất_lượng máy nóng chất_liệutitanium giá rẻ camera iphone chụp_ảnh siêu đẹp ảnh slay liền máy lận máy nóng nâp ios nóng máy tem lazasa check imei sợ lầm hàng hôm_nay cảm tốt pin cục sạc sạc nhanh B-OBJ O B-OBJ O B-ADJ 

In [32]:
train_embeddings = subword2word_embeddings(train_embeddings, sentences_filtered)
test_embeddings = subword2word_embeddings(test_embeddings, test_sentences)


n_samples = len(train_embeddings)
emb_dim = train_embeddings[0].size(-1)  # Lấy kích thước của chiều cuối cùng

pad_id = tokenizer.pad_token_id
pad_id_labels = labels2idx['PAD']

class Batcher:
    def __init__(self, embeddings, labels, batch_size, emb_dim, pad_id, pad_id_labels):
        self.embeddings = embeddings
        self.labels = labels
        self.batch_size = batch_size
        self.emb_dim = emb_dim
        self.pad_id = pad_id
        self.pad_id_labels = pad_id_labels

    def __iter__(self):
        for i in range(0, len(self.embeddings), self.batch_size):
            batch_x = self.embeddings[i:i + self.batch_size]
            batch_y = self.labels[i:i + self.batch_size]

            # Calculate the maximum length in this batch
            max_length_batch = max(x.size(0) for x in batch_x)

            # Pad all tensors to this max length
            padded_batch_x = []
            for x in batch_x:
                pad_length = max_length_batch - x.size(0)
                # Adjust padding to match dimensions of x
                if len(x.size()) == 3:
                    padded_x = torch.cat([x, torch.full((pad_length, x.size(1), self.emb_dim), self.pad_id)], dim=0)
                else:
                    padded_x = torch.cat([x, torch.full((pad_length, self.emb_dim), self.pad_id)], dim=0)
                padded_batch_x.append(padded_x)

            # Stack the padded tensors
            batch_x_final = torch.stack(padded_batch_x)

            # Handle labels similarly if necessary
            padded_labels = [y + [self.pad_id_labels] * (max_length_batch - len(y)) for y in batch_y]
            batch_y_final = torch.tensor(padded_labels, dtype=torch.long)

            yield batch_x_final, batch_y_final

# Example usage of the Batcher class
batcher = Batcher(numpy.asarray(train_embeddings, dtype=object),
                  numpy.asarray(train_labels, dtype=object), 32, emb_dim,
                  pad_id=pad_id, pad_id_labels=pad_id_labels)

model = Classifier(embedding_dim=emb_dim, num_labels=len(labels2idx), dropout=0.01)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for e in range(30):
    total_loss = 0
    for batch in batcher:
        batch_x, batch_y = batch
        optimizer.zero_grad()
        model_output, _ = model(batch_x)
        loss = loss_fn(model_output, batch_y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch:{e}, Loss: {total_loss/n_samples}')

final_predictions = []
final_labels = []
for emb, label in zip(test_embeddings, test_labels):
    _, probs = model(emb)
    predict_labels = torch.argmax(probs, dim=-1)
    final_predictions.extend(predict_labels.tolist())
    final_labels.extend(label)

print(classification_report(final_predictions, final_labels))

C:\Users\HELLO\AppData\Local\Temp\ipykernel_42656\2993054585.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  batcher = Batcher(numpy.asarray(train_embeddings, dtype=object),


MemoryError: Unable to allocate 1.35 MiB for an array with shape (11, 21, 768) and data type object